In [1]:
from tensorflow import keras
import tensorflow as tf
import os

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [3]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [4]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [5]:
X_train_full.shape

(15480, 8)

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [8]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [9]:
keras_reg.fit(X_train, y_train, epochs = 100, validation_data = (X_valid, y_valid), callbacks = [keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)

X_new = X_test[:3]
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4060 - val_l

In [11]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden":(0, 1, 2, 3),
    "n_neurons" : np.arange(1, 100).tolist(),
    "learning_rate" : (3e-4, 3e-2)
    }
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv = 3, verbose=1, n_jobs = 1)
rnd_search_cv.fit(X_train, y_train, 
                  epochs = 100, 
                  validation_data = (X_valid, y_valid), 
                  callbacks = keras.callbacks.EarlyStopping(patience=10), verbose = 1)



Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


242/242 [==============================] - 0s 2ms/step - loss: 0.6502 - val_loss: 35.4030
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7966 - val_loss: 0.7099
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 11/100
242/242 [==========================

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 11.5min finished


363/363 [==============================] - 1s 2ms/step - loss: 0.5662 - val_loss: 0.8724
Epoch 2/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4019 - val_loss: 4.0230
Epoch 3/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4441 - val_loss: 0.4731
Epoch 4/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3904 - val_loss: 11.9244
Epoch 5/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3817 - val_loss: 34.0429
Epoch 6/100
363/363 [==============================] - 1s 1ms/step - loss: 0.6382 - val_loss: 3.5755
Epoch 7/100
363/363 [==============================] - 1s 1ms/step - loss: nan - val_loss: nan
Epoch 8/100
363/363 [==============================] - 1s 1ms/step - loss: nan - val_loss: nan
Epoch 9/100
363/363 [==============================] - 1s 1ms/step - loss: nan - val_loss: nan
Epoch 10/100
363/363 [==============================] - 1s 1ms/step - loss: nan - val_loss: nan
Epoch 11/100
363/363 [=

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000021957464C88>,
                   n_jobs=1,
                   param_distributions={'learning_rate': (0.0003, 0.03),
                                        'n_hidden': (0, 1, 2, 3),
                                        'n_neurons': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...]},
                   verbose=1)

In [12]:
rnd_search_cv.best_params_

{'n_neurons': 59, 'n_hidden': 2, 'learning_rate': 0.03}

The best parameters are {'n_neurons': 59, 'n_hidden': 2, 'learning_rate': 0.03}

In [13]:
rnd_search_cv.best_score_

-0.3011883596579234

In [14]:
model = rnd_search_cv.best_estimator_.model

import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

model.evaluate(X_test, y_test)

X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

import numpy as np

y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

np.array(class_names)[y_pred]

y_test[:3]